In [ ]:
from selenium import webdriver
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import code
from selenium.webdriver.common.action_chains import ActionChains
import time


class YnuLogin(object):
    def __init__(self,driversite,user):
        self.website='http://xk.ynu.edu.cn/xsxkapp/sys/xsxkapp/*default/index.do'
        self.driversite=driversite
        self.username=user['username']
        self.userpassword=user['userpassword']
        self.codename=user['codename']
        self.codepassword=user['codepassword']
        self.codeid=user['codeid']

    # 登录
    def uesrlogin(self):
        chrome_options = Options()
        # 浏览器参数配置
        chrome_options.add_argument('--no-sandbox')  # 解决DevToolsActivePort文件不存在的报错
        chrome_options.add_argument('window-size=1920x3000')  # 指定浏览器分辨率
        chrome_options.add_argument('--disable-gpu')  # 谷歌文档提到需要加上这个属性来规避bug
        chrome_options.add_argument('--hide-scrollbars')  # 隐藏滚动条, 应对一些特殊页面
        # chrome_options.add_argument('--headless')  # 浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败

        self.driver = webdriver.Chrome(executable_path=self.driversite,chrome_options=chrome_options)

        # 打开chrome浏览器
        # driver = webdriver.Chrome(executable_path=driversite,chrome_options=option)
        # driver = webdriver.Chrome(driversite)
        picture = 'pachongmark.png'
        self.driver.get(self.website)
        WebDriverWait(self.driver, 10, 0.5).until(
            EC.presence_of_element_located(
                (By.ID, 'loginName')))
        self.driver.find_element_by_id('loginName').send_keys(self.username)
        self.driver.find_element_by_id('loginPwd').send_keys(self.userpassword)
        infotext='first'
        while infotext!='' and infotext!='请填写验证码':
            WebDriverWait(self.driver, 10, 0.5).until(
                EC.presence_of_element_located(
                    (By.ID, 'vcodeImg')))
            mark = self.driver.find_element_by_id('vcodeImg')
            if infotext!='first':
                mark.click()
            link = mark.get_attribute('src')
            print(link)  # 验证码地址
            response = requests.get(link)
            with open(picture, 'wb') as f:
                f.write(response.content)
            f.close()
            # chaojiying = code.Chaojiying_Client(self.codename, self.codepassword, self.codeid)
            # im = open('pachongmark.png', 'rb')
            # result = chaojiying.PostPic(im.read(), 1902)['pic_str']
            # im.close()
            # print(result)
            result=input()

            vecode=self.driver.find_element_by_id('verifyCode')
            vecode.clear()
            vecode.send_keys(result)
            loginbutton = self.driver.find_element_by_id('studentLoginBtn')
            loginbutton.click()
            info = self.driver.find_element_by_css_selector('#loginDiv > div.cv-btns')
            infotext=info.text
            print(info.text)
        print('登录成功，系统启动')
        WebDriverWait(self.driver, 10, 0.5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#buttons > button.bh-btn.bh-btn.bh-btn-primary.bh-pull-right')))
        sure = self.driver.find_element_by_css_selector('#buttons > button.bh-btn.bh-btn.bh-btn-primary.bh-pull-right')
        sure.click()
        WebDriverWait(self.driver, 10, 0.5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#courseBtn')))
        start = self.driver.find_element_by_css_selector('#courseBtn')
        start.click()
        return self.driver


driversite = 'D:\\code\\chromedriver.exe'  # 浏览器驱动路径
user = {'username': '20171120176', 'userpassword': 'H+142857uang',
        'codename': 'yth123456', 'codepassword': 'h12138', 'codeid': '903265'}


In [98]:
testdriver=YnuLogin(driversite,user).uesrlogin()


http://xk.ynu.edu.cn/xsxkapp/sys/xsxkapp/student/vcode/image.do?vtoken=d6e66027-6bcc-42a6-b0a0-7de097e6bfb4

登录成功，系统启动


In [103]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

class AutoRecommend(object):
    def __init__(self,driver):
        self.driver=driver

    def recommends(self, recommendcondition):
        self.driver.refresh()
        # try:
        #     WebDriverWait(driver, 10, 0.5).until(
        #         EC.presence_of_element_located(
        #             (By.CSS_SELECTOR, '#buttons > button')))
        #     sure=driver.find_element_by_css_selector('#buttons > button')
        #     sure.click()
        # except:
        #     print('sure_error')
        WebDriverWait(self.driver, 10, 0.5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#aPublicCourse')))
        selection = self.driver.find_element_by_css_selector('#aPublicCourse')
        selection.click()
        time.sleep(1)
        result_str = ''
        for e in recommendcondition:
            result_str=result_str+self.run(e)
        return result_str
#cvDialog > div:nth-child(2) > div.cv-foot > div.cv-sure.cvBtnFlag
#cvDialog > div:nth-child(2) > div.cv-foot > div.cv-sure.cvBtnFlag
    def run(self, input_group):
        want_course,want_id = input_group
        selectable_course_list = self.findcourse(want_course)
        message=''
        select_status = 'begin'
        selet_button_place = 0
        for this_course in selectable_course_list:
            text_list = this_course.text.split('\n')
            if len(text_list)>1:
                selet_button_place+=1
                course_id = text_list[0]
                select_peolple=float(text_list[7])/float(text_list[6])
                cssSelect='#publicBody > div:nth-child('+str(selet_button_place)+') > div.cv-setting-col > a'

                if course_id == want_id and select_peolple<1:
                    WebDriverWait(self.driver, 10, 0.5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, cssSelect)))
                    choosebutton = self.driver.find_element_by_css_selector(cssSelect)
                    choosebutton.click()

                    WebDriverWait(self.driver, 10, 0.5).until(
                        EC.presence_of_element_located(
                            (By.CLASS_NAME, 'cvBtnFlag')))
                    surebutton = self.driver.find_element_by_class_name('cvBtnFlag')
                    surebutton.click()

        return message


    def findcourse(self, coursename):
        WebDriverWait(self.driver, 10, 0.5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#publicSearch')))
        inputtext = self.driver.find_element_by_css_selector('#publicSearch')
        time.sleep(0.5)
        inputtext.clear()
        inputtext.send_keys(coursename)
        inputtext.send_keys(Keys.ENTER)
        time.sleep(1)
        selectable_course_list = self.driver.find_elements_by_class_name('cv-row')

        return selectable_course_list


In [104]:
listc=AutoRecommend(testdriver).recommends([('环境','ECOL1I0014[01]')])



['ECOL1I0002[01]', '', '教学班详情', '创新思维训练[已满]', '耿宇鹏', '1-17周 星期一 9-10节 格物楼1栋1507', '60', '60', '创新创业与能力提升', '2', '选择']
#publicBody > div:nth-child(1) > div.cv-setting-col > a
['ECOL1I0003[01]', '', '教学班详情', '环境科学前沿', '王文礼', '1-17周 星期二 9-10节 格物楼1栋1507', '60', '59', '生态文明与运动健康', '2', '选择']
#publicBody > div:nth-child(2) > div.cv-setting-col > a
['ECOL1I0004[02]', '', '教学班详情', '可视化程序设计与未来生活[已满]', '闫海忠', '1-17周 星期二 9-10节 力行楼4509', '60', '60', '创新创业与能力提升', '2', '选择']
#publicBody > div:nth-child(3) > div.cv-setting-col > a
['ECOL1I0005[01]', '', '教学班详情', 'office高级实用技巧[已满]', '张翔', '1-17周 星期二 9-10节 力行楼4510', '60', '60', '创新创业与能力提升', '2', '选择']
#publicBody > div:nth-child(4) > div.cv-setting-col > a
['ECOL1I0008[02]', '', '教学班详情', '3S技术与应用[已满]', '闫海忠', '1-17周 星期一 9-10节 格物楼1栋1505', '60', '60', '科学探索与技术进步', '2', '选择']
#publicBody > div:nth-child(5) > div.cv-setting-col > a
['ECOL1I0010[01]', '', '教学班详情', '生态与环境经济学[已满]', '李俊梅', '1-17周 星期一 9-10节 力行楼3501', '60', '60', '生态文明与运动健康', '2', '选择']
#publicB